# Create translations for contours of DAVIS 2016

In this notebook, the translations between the  points of two consecutive contours are computed and the number of contour points is reduced.

In [20]:
import numpy as np
import os
from scipy import spatial

## Functions

In [22]:
def get_min_indices(distances):
    '''Returns a (N, 2) array containing the indices of the smallest distances
       sorted in ascending order given a (N, N) distance matrix.'''
    
    # Get indices that sort array by minimal distance
    min_indices = np.argsort(distances, axis=None)

    # Turn array of flat indices into a tuple of coordinate arrays
    min_indices = np.unravel_index(min_indices, distances.shape)

    # Turn tuple of arrays into array of coordinate arrays
    min_indices = np.array([min_indices[0], min_indices[1]])

    # Transpose array
    min_indices = min_indices.T

    return min_indices

In [23]:
def get_corresponding_points(contour_0, contour_1):
    '''Returns a (N, 2) array containing the matching of the closest contour 
       points given two contours.'''
    
    # Get distance matrix between every point of contour_0 and contour_1
    distances = spatial.distance.cdist(contour_0, contour_1, 'euclidean')
    
    # Get indices of the smallest distances
    min_indices = get_min_indices(distances)
    
    corresponding_points = np.empty([0, 2], int)
    forbidden_points_0 = set()
    forbidden_points_1 = set()
    
    for min_index in min_indices:
        if ((min_index[0] in forbidden_points_0) or (min_index[1] in forbidden_points_1)):
            continue
        else:
            corresponding_points = np.append(corresponding_points, min_index.reshape([1,2]), axis=0)
            forbidden_points_0.add(min_index[0])
            forbidden_points_1.add(min_index[1])
    
    # Sort corresponding points by first column
    corresponding_points = corresponding_points[corresponding_points[:,0].argsort()]
    
    return corresponding_points

In [28]:
def get_translations(contour_0, contour_1):
    '''Returns the translations for each point in contour_0 to contour_1.'''
    
    corresponding_points = get_corresponding_points(contour_0, contour_1)
    
    translations = np.empty([0, 2], int)
    
    for point in corresponding_points:
        
        translation = np.subtract(contour_1[point[1]], contour_0[point[0]])
        translations = np.append(translations, translation.reshape([1,2]), axis=0)
    
    return translations, corresponding_points 

In [57]:
def create_translations_for_all_contours(contours_folders_path,
                                         translations_folders_path):
    
    contours_folders_list = os.listdir(contours_folders_path)

    # Iterate through folders
    for i, folder in enumerate(contours_folders_list):

        # if (i > 0): break

        print('#{}: {}'.format(i, folder))

        # Create folder to save translations
        translations_folder = os.path.join(translations_folders_path, folder)
        if not os.path.exists(translations_folder):
            os.makedirs(translations_folder)

        # Get list of contours
        contours = os.listdir(os.path.join(contours_folders_path, folder))
        if '.ipynb_checkpoints' in contours:
            contours.remove('.ipynb_checkpoints')
        contours.sort()
            
        # Iterate through annotations
        for j, contour in enumerate(contours):

            # if (j > 0): break
                
            # Load contours
            contour_0_path = os.path.join(contours_folders_path, folder, contours[j])
            contour_0 = np.load(contour_paths[j])
            
            try:
                contour_1_path = os.path.join(contours_folders_path, folder, contours[j+1])
                contour_1 = np.load(contour_paths[j+1])
            except IndexError as e:
                break

            # Compute translations
            translations_0_1, corresponding_points = get_translations(contour_0, contour_1)
            
            # Update contour_1 so that it has same amount of points as contour_0
            corresponding_points = corresponding_points.T 
            contour_1 = contour_1[corresponding_points[1]] 

            # Save contour_1
            np.save(contour_paths[j+1], contour_1)
            
            # Save translations
            np.save(os.path.join(translations_folder, contour[:5]), translations_0_1)

## Create Translations

In [58]:
CONTOURS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Contours/480p'
TRANSLATIONS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Translations/480p'

create_translations_for_all_contours(CONTOURS_FOLDERS_PATH,
                                     TRANSLATIONS_FOLDERS_PATH)

#0: swing
#1: drift-chicane
#2: lucia
#3: soapbox


In [50]:
np.load('DAVIS_2016/DAVIS/Translations/480p/swing/00000.npy')

array([[12,  4],
       [ 8,  5],
       [-5,  6],
       [-5,  2],
       [-7, -6],
       [ 4, -1],
       [11, -2],
       [ 7, -1]])